## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [8]:
EPOCHS=10
NUM_WORDS=10000

## 모델 정의

In [9]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16) # 길이가 32인 숫자들을 길이가 16인 Vector 로 변환
    self.lstm = tf.keras.layers.LSTM(32)
    self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, x, training=None, mask=None):
    x=self.emb(x)
    x=self.lstm(x)
    return self.dense(x)

## IMDB 데이터셋 준비

In [15]:
imdb = tf.keras.datasets.imdb
(x_train, y_train),(x_test,y_test) = imdb.load_data(num_words=NUM_WORDS)
# print(len(x_train[0]), len(x_train[1]))
# 32길이로 잘라주고, 32가 안될경우 0으로 패딩해준다.
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)

x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

## 모델생성

In [14]:

model=MyModel()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 학습 루프 동작

In [13]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 14s 18ms/step - loss: 0.2556 - val_loss: 0.5243
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.2083 - val_loss: 0.6053
Epoch 3/10
782/782 [==============================] - 14s 18ms/step - loss: 0.1658 - val_loss: 0.7357
Epoch 4/10
782/782 [==============================] - 14s 18ms/step - loss: 0.1316 - val_loss: 0.7740
Epoch 5/10
782/782 [==============================] - 14s 18ms/step - loss: 0.1023 - val_loss: 1.0966
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0884 - val_loss: 1.1194
Epoch 7/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0732 - val_loss: 1.1734
Epoch 8/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0581 - val_loss: 1.5046
Epoch 9/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0486 - val_loss: 1.4008
Epoch 10/10
782/782 [==============================] - 14s 18ms/step - loss: 0.040

##  모델 학습결과
- loss 감소하지만 val_loss 가 증가하는 것을 알 수 있다
- LSTM 대신 GRU / RNN 사용해볼 것